In [1]:
# Import necessary libraries
import random, time, os
import mysql.connector
from dotenv import load_dotenv
from Problem1 import hash_join

load_dotenv()

Join Output:
(1, 10, 101)
(2, 20, 102)
(3, 30, 103)
(4, 40, 104)
(5, 50, 105)
(6, 60, 106)
(7, 70, 107)
(8, 80, 108)
(9, 90, 109)
(10, 100, 110)


True

# Problem 2 implementation: Simplified version of Yannakakis' algorithm

In [2]:
def construct_hashmap_problem2(arr, index):
    """
    Docstring for construct_hashmap_problem2
    
    :param arr: a 2D array
    :param index: the index to be used as key. Only 0 or 1.
    :return: a hashmap where the keys are the elements at the given index.
    """
    h = {}
    for i in range(len(arr)):
        cur_element = arr[i]
        key = cur_element[index]
        val = cur_element[1 - index]
        if key not in h:
            h[key] = [val]
        else:
            h[key].append(val)
    return h

def remove_dangling_tuple(db):
    """
    Docstring for remove_dangling_tuple
    
    :param db: a 3D array. Each element is a 2D array representing a table.
    """
    h_bottom_up = {i: construct_hashmap_problem2(db[i], 1) for i in range(len(db))}
    h_top_down = {i: construct_hashmap_problem2(db[i], 0) for i in range(len(db))}
    
    # Bottom up pass
    for i in range(len(db)-2, -1, -1):
        for j in range(len(db[i])):
            cur_ai1 = db[i][j][0]
            cur_ai2 = db[i][j][1]
            if h_top_down[i+1].get(cur_ai2) is None:
                h_bottom_up[i][cur_ai2].remove(cur_ai1)
                if len(h_bottom_up[i][cur_ai2]) == 0:
                    del h_bottom_up[i][cur_ai2]

    # Top down pass is not necessary because when we obtain the result by going top down,
    # the dangling tuples would not be included anyway. The top down pass is automatically
    # handled in get_result function.
    # for i in range(1, len(db)):
    #     for j in range(len(db[i])):
    #         cur_ai1 = db[i][j][0]
    #         cur_ai2 = db[i][j][1]
    #         if h_bottom_up[i-1].get(cur_ai1) is None:
    #             h_top_down[i][cur_ai1].remove(cur_ai2)
    #             if len(h_top_down[i][cur_ai1]) == 0:
    #                 del h_top_down[i][cur_ai1]
                
    return h_bottom_up, h_top_down

def get_result(h_top_down):
    """
    Docstring for get_result
    
    :param h_top_down: a hashmap from remove_dangling_tuple function. This is used by the DFS to get the result of the query.
    """
    result = []
    def dfs(i, current_tuple, val):
        # If we have reached the end, append the current tuple to result
        if i == len(h_top_down):
            for j in current_tuple:
                result.append(j + val)
            return
        
        # If we are at the first relation, we need to initialize the current_tuple
        # and call dfs for the next relation
        if i == 0:
            for key in h_top_down[i]:
                for tup in h_top_down[i][key]:
                    dfs(i+1, [[key]], [tup])
                    
        # Else, append to existing values to current_tuple and call dfs for the next relation
        else:
            for key in val:
                if key in h_top_down[i]:
                    for tup in h_top_down[i][key]:
                        dfs(i+1, [i + val for i in current_tuple], [tup])                    
    
    dfs(0, [], [])
    return result

In [3]:
# Test case
R1 = [[1, 10], [2, 20], [3, 30]]
R2 = [[10, 100], [20, 200], [99, 999], [30, 200]]   # (99,999) is dangling
R3 = [[100, 1000], [200, 2000]]

relations = [R1, R2, R3]

h1, h2 = remove_dangling_tuple(relations)
result = get_result(h2)
for r in result:
    print(r)

[1, 10, 100, 1000]
[2, 20, 200, 2000]
[3, 30, 200, 2000]


# Problem 3: Left to right implementation
We implement two versions of this algorithm. The first utilizes hashmaps to speed up key looking up process. The second perform nested loops to perform the join.

In [4]:
# Problem 3 implementation hashmap: This implementation is directly join the line query from left to right using hashmap.
def problem3_algo(db):
    """
    Docstring for problem3_algo
    
    :param db: database as 3D array. Each element is a 2D array representing a relation. Assume that there are at least 2 relations.
    """
    current_result = hash_join(db[0], db[1])
    current_result = [list(i) for i in current_result] # convert tuple to list for consistency
    for i in range(2, len(db)):
        temp_result = []
        h = {}
        for (b, c) in db[i]:
            if b not in h:
                h[b] = []
            h[b].append((b, c))
        
        for data in current_result:
            joining_attribute = data[-1]
            if joining_attribute in h:
                for (_, c) in h[joining_attribute]:
                    temp_result.append(data + [c])
        
        current_result = temp_result
    
    return current_result

In [5]:
# Test case for hashmap implementation
result = problem3_algo(relations)
for r in result:
    print(r)

[1, 10, 100, 1000]
[2, 20, 200, 2000]
[3, 30, 200, 2000]


# Problem 4: Build test database to compare run time.

In [6]:
# Problem 4:
R1 = []
for i in range(1, 101):
    R1.append([i, random.randint(1, 5000)])

R2 = []
for i in range(1, 101):
    R2.append([random.randint(1, 5000), i])
    
R3 = []
for i in range(1, 101):
    R3.append([i, i])
    
db = [R1, R2, R3]

In [7]:
start_time = time.perf_counter()
h1, h2 = remove_dangling_tuple(db)
result = get_result(h2)
print("Results from Problem 2's implementation:")
if not result:
    print("No result found.")
else:
    for r in result:
        print(r)
end_time = time.perf_counter()
print(f"Execution time for problem 2's implementation: {end_time - start_time} seconds")

Results from Problem 2's implementation:
[86, 1179, 99, 99]
Execution time for problem 2's implementation: 0.0006741000106558204 seconds


In [8]:
start_time = time.perf_counter()
result = problem3_algo(db)
print("Results from Problem 3's implementation:")
if not result:
    print("No result found.")
else:
    for r in result:
        print(r)
end_time = time.perf_counter()
print(f"Execution time for problem 3's implementation: {end_time - start_time} seconds")

Results from Problem 3's implementation:
[86, 1179, 99, 99]
Execution time for problem 3's implementation: 0.0003106000367552042 seconds


# Problem 5: build test database to compare run time.

In [9]:
# Problem 5 implementation:
R1 = []

for i in range(1, 1001):
    R1.append([i, 5])
for i in range(1001, 2001):
    R1.append([i, 7])
R1.append([2001, 2002])
random.shuffle(R1)


R2 = []
for i in range(1, 1001):
    R2.append([5, i])
for i in range(1001, 2001):
    R2.append([7, i])
R2.append([2002, 8])
random.shuffle(R2)

R3 = []
for i in range(2000):
    R3.append([random.randint(2002, 3000), random.randint(1, 3000)])
R3.append([8, 30])
random.shuffle(R3)

db = [R1, R2, R3]

In [10]:
start_time = time.perf_counter()
h1, h2 = remove_dangling_tuple(db)
result = get_result(h2)
print("Results from Problem 2's implementation:")
if not result:
    print("No result found.")
else:
    for r in result:
        print(r)
end_time = time.perf_counter()
print(f"Execution time for problem 2's implementation: {end_time - start_time} seconds")

Results from Problem 2's implementation:
[248, 5, 8, 30]
[287, 5, 8, 30]
[978, 5, 8, 30]
[703, 5, 8, 30]
[788, 5, 8, 30]
[513, 5, 8, 30]
[926, 5, 8, 30]
[663, 5, 8, 30]
[72, 5, 8, 30]
[454, 5, 8, 30]
[83, 5, 8, 30]
[994, 5, 8, 30]
[695, 5, 8, 30]
[337, 5, 8, 30]
[108, 5, 8, 30]
[628, 5, 8, 30]
[402, 5, 8, 30]
[2, 5, 8, 30]
[570, 5, 8, 30]
[621, 5, 8, 30]
[25, 5, 8, 30]
[859, 5, 8, 30]
[935, 5, 8, 30]
[797, 5, 8, 30]
[504, 5, 8, 30]
[899, 5, 8, 30]
[752, 5, 8, 30]
[684, 5, 8, 30]
[212, 5, 8, 30]
[29, 5, 8, 30]
[84, 5, 8, 30]
[372, 5, 8, 30]
[567, 5, 8, 30]
[357, 5, 8, 30]
[49, 5, 8, 30]
[123, 5, 8, 30]
[397, 5, 8, 30]
[408, 5, 8, 30]
[138, 5, 8, 30]
[532, 5, 8, 30]
[260, 5, 8, 30]
[566, 5, 8, 30]
[168, 5, 8, 30]
[455, 5, 8, 30]
[713, 5, 8, 30]
[817, 5, 8, 30]
[741, 5, 8, 30]
[188, 5, 8, 30]
[31, 5, 8, 30]
[641, 5, 8, 30]
[557, 5, 8, 30]
[656, 5, 8, 30]
[690, 5, 8, 30]
[236, 5, 8, 30]
[467, 5, 8, 30]
[356, 5, 8, 30]
[342, 5, 8, 30]
[254, 5, 8, 30]
[921, 5, 8, 30]
[657, 5, 8, 30]
[59, 5, 

In [11]:
start_time = time.perf_counter()
result = problem3_algo(db)
print("Results from Problem 3's implementation:")
if not result:
    print("No result found.")
else:
    for r in result:
        print(r)
end_time = time.perf_counter()
print(f"Execution time for problem 3's implementation: {end_time - start_time} seconds")

Results from Problem 3's implementation:
[248, 5, 8, 30]
[287, 5, 8, 30]
[978, 5, 8, 30]
[703, 5, 8, 30]
[788, 5, 8, 30]
[513, 5, 8, 30]
[926, 5, 8, 30]
[663, 5, 8, 30]
[72, 5, 8, 30]
[454, 5, 8, 30]
[83, 5, 8, 30]
[994, 5, 8, 30]
[695, 5, 8, 30]
[337, 5, 8, 30]
[108, 5, 8, 30]
[628, 5, 8, 30]
[402, 5, 8, 30]
[2, 5, 8, 30]
[570, 5, 8, 30]
[621, 5, 8, 30]
[25, 5, 8, 30]
[859, 5, 8, 30]
[935, 5, 8, 30]
[797, 5, 8, 30]
[504, 5, 8, 30]
[899, 5, 8, 30]
[752, 5, 8, 30]
[684, 5, 8, 30]
[212, 5, 8, 30]
[29, 5, 8, 30]
[84, 5, 8, 30]
[372, 5, 8, 30]
[567, 5, 8, 30]
[357, 5, 8, 30]
[49, 5, 8, 30]
[123, 5, 8, 30]
[397, 5, 8, 30]
[408, 5, 8, 30]
[138, 5, 8, 30]
[532, 5, 8, 30]
[260, 5, 8, 30]
[566, 5, 8, 30]
[168, 5, 8, 30]
[455, 5, 8, 30]
[713, 5, 8, 30]
[817, 5, 8, 30]
[741, 5, 8, 30]
[188, 5, 8, 30]
[31, 5, 8, 30]
[641, 5, 8, 30]
[557, 5, 8, 30]
[656, 5, 8, 30]
[690, 5, 8, 30]
[236, 5, 8, 30]
[467, 5, 8, 30]
[356, 5, 8, 30]
[342, 5, 8, 30]
[254, 5, 8, 30]
[921, 5, 8, 30]
[657, 5, 8, 30]
[59, 5, 

# Problem 6: Compare with running time in MySQL

In [12]:
mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password=os.getenv("PASSWORDSQL"),
      database="cs580final"
    )

In [13]:
mycursor = mydb.cursor()

In [14]:
# Create tables. Commented out to avoid re-creation errors.
# mycursor.execute("CREATE TABLE R1 (a1 INT NOT NULL, a2 INT NOT NULL);")
# mycursor.execute("CREATE TABLE R2 (a2 INT NOT NULL, a3 INT NOT NULL);")
# mycursor.execute("CREATE TABLE R3 (a3 INT NOT NULL, a4 INT NOT NULL);")

In [15]:
# Insert data into R1. Commented out to avoid duplicate entries.
empty_query = "TRUNCATE TABLE R1;"
insert_query = "INSERT INTO R1 (a1, a2) VALUES (%s, %s)"

mycursor.execute(empty_query)
mydb.commit()

mycursor.executemany(insert_query, R1)
mydb.commit()

In [16]:
# Insert data into R2.
empty_query = "TRUNCATE TABLE R2;"
insert_query = "INSERT INTO R2 (a2, a3) VALUES (%s, %s)"

mycursor.execute(empty_query)
mydb.commit()

mycursor.executemany(insert_query, R2)
mydb.commit()

In [17]:
# Insert data into R3.
empty_query = "TRUNCATE TABLE R3;"
insert_query = "INSERT INTO R3 (a3, a4) VALUES (%s, %s)"

mycursor.execute(empty_query)
mydb.commit()

mycursor.executemany(insert_query, R3)
mydb.commit()

In [18]:
start_time = time.perf_counter()
mycursor.execute("SELECT r1.a1, r2.a2, r3.a3, r3.a4 FROM r1 join r2 on r1.a2 = r2.a2 join r3 on r2.a3 = r3.a3;")
end_time = time.perf_counter()
print(f"Execution time for MySQL: {end_time - start_time} seconds")
myresult = mycursor.fetchall()
print("Results from MySQL:")
for r in myresult:
    print(r)

Execution time for MySQL: 0.6796792999375612 seconds
Results from MySQL:
(2001, 2002, 8, 30)
(248, 5, 8, 30)
(287, 5, 8, 30)
(978, 5, 8, 30)
(703, 5, 8, 30)
(788, 5, 8, 30)
(513, 5, 8, 30)
(926, 5, 8, 30)
(663, 5, 8, 30)
(72, 5, 8, 30)
(454, 5, 8, 30)
(83, 5, 8, 30)
(994, 5, 8, 30)
(695, 5, 8, 30)
(337, 5, 8, 30)
(108, 5, 8, 30)
(628, 5, 8, 30)
(402, 5, 8, 30)
(2, 5, 8, 30)
(570, 5, 8, 30)
(621, 5, 8, 30)
(25, 5, 8, 30)
(859, 5, 8, 30)
(935, 5, 8, 30)
(797, 5, 8, 30)
(504, 5, 8, 30)
(899, 5, 8, 30)
(752, 5, 8, 30)
(684, 5, 8, 30)
(212, 5, 8, 30)
(29, 5, 8, 30)
(84, 5, 8, 30)
(372, 5, 8, 30)
(567, 5, 8, 30)
(357, 5, 8, 30)
(49, 5, 8, 30)
(123, 5, 8, 30)
(397, 5, 8, 30)
(408, 5, 8, 30)
(138, 5, 8, 30)
(532, 5, 8, 30)
(260, 5, 8, 30)
(566, 5, 8, 30)
(168, 5, 8, 30)
(455, 5, 8, 30)
(713, 5, 8, 30)
(817, 5, 8, 30)
(741, 5, 8, 30)
(188, 5, 8, 30)
(31, 5, 8, 30)
(641, 5, 8, 30)
(557, 5, 8, 30)
(656, 5, 8, 30)
(690, 5, 8, 30)
(236, 5, 8, 30)
(467, 5, 8, 30)
(356, 5, 8, 30)
(342, 5, 8, 30)
(254

In [19]:
mycursor.execute("EXPLAIN SELECT r1.a1, r2.a2, r3.a3, r3.a4 FROM r1 join r2 on r1.a2 = r2.a2 join r3 on r2.a3 = r3.a3;")
myresult = mycursor.fetchall()
print("Query Execution Plan from MySQL:")
for r in myresult:
    print(r)

Query Execution Plan from MySQL:
(1, 'SIMPLE', 'r1', None, 'ALL', None, None, None, None, 1996, 100.0, None)
(1, 'SIMPLE', 'r2', None, 'ALL', None, None, None, None, 2001, 10.0, 'Using where; Using join buffer (hash join)')
(1, 'SIMPLE', 'r3', None, 'ALL', None, None, None, None, 2001, 10.0, 'Using where; Using join buffer (hash join)')
